## **Set directories**

In [5]:
import os

tools_path = "./tools"
assert os.path.exists(tools_path), f"tools path does not exists: {tools_path}"

DATASETS = "./data"
dtd_path = DATASETS + "/dtd"
mvtec_path = DATASETS + "/mvtec"
assert os.path.exists(dtd_path), f"DTD path does not exists: {dtd_path}"
assert os.path.exists(mvtec_path), f"MVTEC path does not exists: {mvtec_path}"

OUTPUT_PATH = "./result"
assert os.path.exists(OUTPUT_PATH), f"DTD path does not exists: {OUTPUT_PATH}"


print("Directories set.")

Directories set.


## **Install the library required**

In [ ]:
%pip install -r {WORKDIR}/requirements.txt

## **Generate foreground and global retrieval result**

In [6]:
!python {tools_path}/generate_foreground.py \
    -lp {OUTPUT_PATH}/foreground \
    --data_dir {DATASETS} \
    --vis

!python {tools_path}/generate_retrieval.py \
    -lp {OUTPUT_PATH}/retrieval \
    --data_dir {DATASETS} \
    --vis \
    -k 5


2025/03/04 05:00:46 INFO mlflow.tracking.fluent: Experiment with name 'CPR_foreground' does not exist. Creating a new experiment.
2025-03-04 05:00:46.042 | INFO     | __main__:<module>:97 - args: 
{'data_dir': './data',
 'dataset_name': 'mvtec',
 'layer': 'features.denseblock1',
 'log_path': './result/foreground',
 'pretrained_model': 'DenseNet',
 'resize': 320,
 'vis': True}
2025-03-04 05:00:49.462 | INFO     | __main__:gen_foreground:26 - gen_foreground
2025-03-04 05:00:49.464 | INFO     | __main__:gen_foreground:27 - save to ./result/foreground
2025-03-04 05:00:49.464 | INFO     | __main__:gen_foreground:28 - params: mvtec DenseNet features.denseblock1 320 True
2025-03-04 05:00:51.183 | INFO     | __main__:gen_foreground:35 - generate bottle
2025-03-04 05:00:59.165 | INFO     | __main__:gen_foreground:52 - predict foreground
2025-03-04 05:01:24.164 | INFO     | __main__:gen_foreground:35 - generate cable
2025-03-04 05:01:33.161 | INFO     | __main__:gen_foreground:52 - predict foreg

## **Train**

### ***Generate synthetic data***

In [9]:
!python {tools_path}/generate_synthetic_data.py \
    -fd {OUTPUT_PATH}/foreground \
    -lp {OUTPUT_PATH}/synthetic_mvtec

2025/03/04 05:16:21 INFO mlflow.tracking.fluent: Experiment with name 'CPR_synthetic' does not exist. Creating a new experiment.
2025-03-04 05:16:21.901 | INFO     | __main__:<module>:178 - args: 
{'dataset_name': 'mvtec',
 'foreground_dir': './result/foreground',
 'log_path': './result/synthetic_mvtec',
 'num': 12000,
 'num_workers': 12,
 'resize': 640}
2025-03-04 05:16:22.455 | INFO     | __main__:generate:137 - gen_simulated_anomaly
2025-03-04 05:16:22.456 | INFO     | __main__:generate:138 - save to ./result/synthetic_mvtec
2025-03-04 05:16:22.456 | INFO     | __main__:generate:139 - params: 12 mvtec 640 12000 ./result/foreground
2025-03-04 05:16:22.456 | INFO     | __main__:generate:144 - processing bottle
[ WARN:0@7.318] global loadsave.cpp:848 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_8U.
[ WARN:0@9.075] global loadsave.cpp:848 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_8U.
[ WARN:0@9.357] global loadsave.cpp:848 imw

### ***Run***

*Please set the OUTPUT_PATH which is same as before*

In [5]:
%%bash

# objects=('bottle' 'cable' 'capsule' 'hazelnut' 'metal_nut' 'pill' 'screw' 'toothbrush' 'transistor' 'zipper')
# textures=('carpet' 'grid' 'leather' 'tile' 'wood')
objects=('bottle')
OUTPUT_PATH="./result"

# [10] bottle test result: pro: 0.9593 ap: 0.8097 pixel-auc: 0.9819 image-auc: 1.0000
# [20] bottle test result: pro: 0.9619 ap: 0.8193 pixel-auc: 0.9836 image-auc: 1.0000
# [50] bottle test result: pro: 0.9701 ap: 0.8754 pixel-auc: 0.9872 image-auc: 1.0000

# python train.py -lp log/mvtec_train --sub-categories "${textures[@]}" \
# -fd ./log/foreground_mvtec_DenseNet_features.denseblock1_320/ \
# --steps 500 \
# -tps 50 \
# --data-dir log/synthetic_mvtec_640_12000_True_jpg

python train.py \
    -lp ${OUTPUT_PATH}/mvtec_train \
    --sub-categories "${objects[@]}" \
    -fd ${OUTPUT_PATH}/foreground/ \
    -rd ${OUTPUT_PATH}/retrieval \
    --steps 2 \
    -tps 1 \
    --data-dir ${OUTPUT_PATH}/synthetic_mvtec

2025-03-05 04:41:14.535 | INFO     | __main__:main:100 - run argv: train.py -lp ./result/mvtec_train --sub-categories bottle -fd ./result/foreground/ -rd ./result/retrieval --steps 2 -tps 1 --data-dir ./result/synthetic_mvtec
2025-03-05 04:41:14.536 | INFO     | __main__:main:101 - args: 
{'T': 512,
 'batch_size': 32,
 'data_dir': './result/synthetic_mvtec',
 'dataset_name': 'mvtec',
 'foreground_dir': './result/foreground/',
 'k_nearest': 10,
 'learning_rate': 0.001,
 'log_path': './result/mvtec_train',
 'n_anomaly': 0,
 'num_workers': 8,
 'pretrained_model': 'DenseNet',
 'region_sizes': [3, 1],
 'resize': 320,
 'retrieval_dir': './result/retrieval',
 'scales': [4, 8],
 'steps': 2,
 'sub_categories': ['bottle'],
 'test_per_steps': 1}


2025/03/05 04:41:14 INFO mlflow.tracking.fluent: Experiment with name 'CPR_mvtec' does not exist. Creating a new experiment.
INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/densenet201.tv_in1k)
INFO:timm.models._hub:[timm/densenet201.tv_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
extract train local features:  99%|█████████▉| 207/209 [00:08<00:00, 30.16it/s]
                                                                               
predict test data: 100%|██████████| 83/83 [00:08<00:00, 10.46it/s]


preds: [tensor(34.5944, device='cuda:0', dtype=torch.float64), tensor(39.2016, device='cuda:0', dtype=torch.float64), tensor(34.0952, device='cuda:0', dtype=torch.float64), tensor(38.2505, device='cuda:0', dtype=torch.float64), tensor(42.4064, device='cuda:0', dtype=torch.float64), tensor(37.8859, device='cuda:0', dtype=torch.float64), tensor(46.7554, device='cuda:0', dtype=torch.float64), tensor(32.5379, device='cuda:0', dtype=torch.float64), tensor(40.0868, device='cuda:0', dtype=torch.float64), tensor(32.0481, device='cuda:0', dtype=torch.float64), tensor(35.8527, device='cuda:0', dtype=torch.float64), tensor(32.8104, device='cuda:0', dtype=torch.float64), tensor(40.9483, device='cuda:0', dtype=torch.float64), tensor(30.0277, device='cuda:0', dtype=torch.float64), tensor(44.4660, device='cuda:0', dtype=torch.float64), tensor(37.3929, device='cuda:0', dtype=torch.float64), tensor(41.5388, device='cuda:0', dtype=torch.float64), tensor(42.9545, device='cuda:0', dtype=torch.float64), te

                                                                  /home/qjh/anaconda3/envs/cpr/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)


2025-03-05 04:41:52.449 | INFO     | __main__:main:155 - [1] bottle test result: pro: nan ap: nan pixel-auc: 0.0000 image-auc: 1.0000 image-f1: 1.0000 f1-threshold: 1.0000


extract train local features:  99%|█████████▉| 207/209 [00:06<00:00, 32.48it/s]
                                                                               
predict test data: 100%|██████████| 83/83 [00:08<00:00, 10.58it/s]


preds: [tensor(42.7859, device='cuda:0', dtype=torch.float64), tensor(51.9225, device='cuda:0', dtype=torch.float64), tensor(43.8092, device='cuda:0', dtype=torch.float64), tensor(48.9353, device='cuda:0', dtype=torch.float64), tensor(56.2505, device='cuda:0', dtype=torch.float64), tensor(49.8612, device='cuda:0', dtype=torch.float64), tensor(61.3782, device='cuda:0', dtype=torch.float64), tensor(43.1786, device='cuda:0', dtype=torch.float64), tensor(51.6970, device='cuda:0', dtype=torch.float64), tensor(40.9411, device='cuda:0', dtype=torch.float64), tensor(47.4981, device='cuda:0', dtype=torch.float64), tensor(42.0424, device='cuda:0', dtype=torch.float64), tensor(55.2100, device='cuda:0', dtype=torch.float64), tensor(37.3463, device='cuda:0', dtype=torch.float64), tensor(56.9627, device='cuda:0', dtype=torch.float64), tensor(48.4702, device='cuda:0', dtype=torch.float64), tensor(56.2298, device='cuda:0', dtype=torch.float64), tensor(57.8981, device='cuda:0', dtype=torch.float64), te

## **Test**

In [8]:
!python test.py \
    -lp result/mvtec_test \
    -fd result/foreground/ \
    -rd result/retrieval \
    --checkpoints weights/{category}.pth

INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/densenet201.tv_in1k)
INFO:timm.models._hub:[timm/densenet201.tv_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
preds: [tensor(929.4794, device='cuda:0', dtype=torch.float64), tensor(894.8674, device='cuda:0', dtype=torch.float64), tensor(892.0562, device='cuda:0', dtype=torch.float64), tensor(1007.4257, device='cuda:0', dtype=torch.float64), tensor(856.0461, device='cuda:0', dtype=torch.float64), tensor(850.3935, device='cuda:0', dtype=torch.float64), tensor(925.3005, device='cuda:0', dtype=torch.float64), tensor(986.9723, device='cuda:0', dtype=torch.float64), tensor(938.2934, device='cuda:0', dtype=torch.float64), tensor(745.2186, device='cuda:0', dtype=torch.float64), tensor(926.6234, device='cuda:0', dtype=torch.float64), tensor(798.9389, device='cuda:0', dtype=torch.float64), tensor(852.0171, device='cuda:0', dtype=torch.float64), 

## **Open the mlflow ui**

In [1]:
import mlflow

!mlflow ui --backend-store-uri file:'./mlruns' 

[2025-03-05 17:04:54 +0800] [5909] [INFO] Starting gunicorn 23.0.0
[2025-03-05 17:04:54 +0800] [5909] [INFO] Listening at: http://127.0.0.1:5000 (5909)
[2025-03-05 17:04:54 +0800] [5909] [INFO] Using worker: sync
[2025-03-05 17:04:54 +0800] [5910] [INFO] Booting worker with pid: 5910
[2025-03-05 17:04:54 +0800] [5911] [INFO] Booting worker with pid: 5911
[2025-03-05 17:04:54 +0800] [5912] [INFO] Booting worker with pid: 5912
[2025-03-05 17:04:54 +0800] [5913] [INFO] Booting worker with pid: 5913
[2025-03-05 17:06:22 +0800] [5909] [INFO] Handling signal: int
^C

Aborted!
[2025-03-05 17:06:22 +0800] [5910] [INFO] Worker exiting (pid: 5910)
[2025-03-05 17:06:22 +0800] [5911] [INFO] Worker exiting (pid: 5911)
[2025-03-05 17:06:22 +0800] [5913] [INFO] Worker exiting (pid: 5913)
[2025-03-05 17:06:22 +0800] [5912] [INFO] Worker exiting (pid: 5912)
